# 🚀 Thai Text Classification with Transformer
### Encoder: thai2vec | Decoder: Transformer
---
**Settings:**
- MAX_LEN = 256
- BATCH_SIZE = 128
- EPOCHS = 500

In [8]:
# 📦 Install dependencies
!pip install -q transformers scikit-learn pythainlp pandas gensim x-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 98.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.0 MB/s eta 0:00:00


In [9]:
# 📚 Import libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from pythainlp import word_tokenize
from pythainlp import word_vector
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from x_transformers import TransformerWrapper, Decoder
import time
import os

In [ ]:
# 🔧 Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🟢 Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# ⚙️ Settings
MAX_LEN = 256
BATCH_SIZE = 128
EPOCHS = 500
LEARNING_RATE = 1e-4
CHECKPOINT_EVERY = 10  # บันทึก checkpoint ทุก 10 epochs

In [ ]:
# 📂 Load Data
# ⚠️ Upload ไฟล์ prachatai_train.csv ไปที่ Kaggle Input ก่อน!
# แก้ path ตามที่ upload

# สำหรับ Kaggle:
# df = pd.read_csv("/kaggle/input/your-dataset/prachatai_train.csv")

# สำหรับ Colab (upload ไฟล์ก่อน):
# from google.colab import files
# uploaded = files.upload()
# df = pd.read_csv("prachatai_train.csv")

df = pd.read_csv("/kaggle/input/prachatai-train/prachatai_train.csv")

texts = df["body_text"].astype(str).tolist()
label_cols = [
    "politics", "human_rights", "quality_of_life", "international",
    "social", "environment", "economics", "culture", "labor",
    "national_security", "ict", "education"
]
y = df[label_cols].values.astype(np.float32)

print(f"📊 Dataset size: {len(texts)}")
print(f"📋 Labels: {label_cols}")

In [ ]:
# 🔤 Load thai2vec Word Embedding
print("กำลังโหลด thai2fit_wv...")
model_wv = word_vector.WordVector(model_name="thai2fit_wv").get_model()
w2v = model_wv
embedding_dim = w2v.vector_size

print(f"✅ Embedding dimension: {embedding_dim}")

# Tokenize
print("กำลัง tokenize...")
tokenized_texts = [word_tokenize(t, keep_whitespace=False) for t in texts]

# Build vocabulary
w2v_vocab = list(w2v.key_to_index.keys())
vocab = {"<PAD>": 0, "<UNK>": 1}
for i, word in enumerate(w2v_vocab, start=2):
    vocab[word] = i

print(f"✅ Vocab size: {len(vocab)}")

In [ ]:
# 🔧 Encode & Pad
def encode_text(tokens, vocab):
    return [vocab.get(w, vocab["<UNK>"]) for w in tokens]

def pad_sequences(sequences, max_len=256, pad_value=0):
    padded = np.full((len(sequences), max_len), pad_value, dtype=np.int64)
    lengths = np.array([min(len(seq), max_len) for seq in sequences], dtype=np.int64)
    for i, seq in enumerate(sequences):
        padded[i, :min(len(seq), max_len)] = seq[:max_len]
    return padded, lengths

encoded_texts = [encode_text(tokens, vocab) for tokens in tokenized_texts]
X, lengths = pad_sequences(encoded_texts, max_len=MAX_LEN)

print(f"✅ X shape: {X.shape}")

In [ ]:
# 📊 Train/Test Split
X_train, X_test, y_train, y_test, len_train, len_test = train_test_split(
    X, y, lengths, test_size=0.1, random_state=42
)

print(f"✅ Train size: {len(X_train)}")
print(f"✅ Test size: {len(X_test)}")

In [ ]:
# 📦 Dataset & DataLoader
class ThaiTextDataset(Dataset):
    def __init__(self, X, lengths, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.lengths = torch.tensor(lengths, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.lengths[idx], self.y[idx]

train_dataset = ThaiTextDataset(X_train, len_train, y_train)
test_dataset = ThaiTextDataset(X_test, len_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                          num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, 
                         num_workers=2, pin_memory=True)

print(f"✅ Batches per epoch: {len(train_loader)}")

In [ ]:
# 🧠 Build Embedding Matrix
vocab_size = max(vocab.values()) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, idx in vocab.items():
    if word in w2v:
        embedding_matrix[idx] = w2v[word]
    elif word == "<PAD>":
        embedding_matrix[idx] = np.zeros(embedding_dim)
    else:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

print(f"✅ Embedding matrix shape: {embedding_matrix.shape}")

In [ ]:
# 🤖 Transformer Model
class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, max_seq_len, output_dim, 
                 depth=4, heads=4, embedding_matrix=None):
        super().__init__()
        
        self.transformer = TransformerWrapper(
            num_tokens=vocab_size,
            max_seq_len=max_seq_len,
            attn_layers=Decoder(  # 👈 เปลี่ยน Encoder เป็น Decoder!
                dim=embed_dim,
                depth=depth,
                heads=heads,
                attn_dropout=0.1,
                ff_dropout=0.1
            )
        )
        
        if embedding_matrix is not None:
            self.transformer.token_emb.emb.weight.data.copy_(
                torch.tensor(embedding_matrix, dtype=torch.float32)
            )
        
        self.fc1 = nn.Linear(embed_dim, embed_dim // 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.fc2 = nn.Linear(embed_dim // 2, output_dim)

    def forward(self, text, lengths=None):
        transformer_out = self.transformer(text, return_embeddings=True)
        
        if lengths is not None:
            mask = torch.arange(text.size(1), device=text.device).unsqueeze(0) < lengths.unsqueeze(1)
            mask = mask.unsqueeze(-1).float()
            pooled = (transformer_out * mask).sum(dim=1) / mask.sum(dim=1)
        else:
            pooled = transformer_out.mean(dim=1)
        
        out = self.fc1(pooled)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        
        return out

In [ ]:
# 🏗️ Create Model
output_dim = len(label_cols)

model = TransformerClassifier(
    vocab_size=len(vocab),
    embed_dim=embedding_dim,
    max_seq_len=MAX_LEN,
    output_dim=output_dim,
    depth=2,
    heads=6,
    embedding_matrix=embedding_matrix
).to(device)

print(f"✅ Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# 🎯 Training with Mixed Precision
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = GradScaler()  # Mixed Precision

best_loss = float('inf')
start_epoch = 0

# 📂 Resume from checkpoint if exists
checkpoint_path = 'checkpoint.pth'
if os.path.exists(checkpoint_path):
    print("📂 Loading checkpoint...")
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    best_loss = checkpoint['best_loss']
    print(f"✅ Resumed from epoch {start_epoch}")

print(f"\n🚀 Starting training from epoch {start_epoch + 1}...")
total_start_time = time.time()

for epoch in range(start_epoch, EPOCHS):
    epoch_start_time = time.time()
    model.train()
    total_loss = 0
    
    for batch_idx, (X_batch, lengths_batch, y_batch) in enumerate(train_loader):
        X_batch = X_batch.to(device)
        lengths_batch = lengths_batch.to(device)
        y_batch = y_batch.to(device)
        
        optimizer.zero_grad()
        
        # Mixed Precision Forward
        with autocast():
            outputs = model(X_batch, lengths_batch)
            loss = criterion(outputs, y_batch)
        
        # Mixed Precision Backward
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
        
        # Progress bar
        if (batch_idx + 1) % 50 == 0:
            print(f"  Batch {batch_idx + 1}/{len(train_loader)}, Loss: {loss.item():.4f}", end='\r')
    
    scheduler.step()
    avg_loss = total_loss / len(train_loader)
    epoch_time = time.time() - epoch_start_time
    
    # Save best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), 'best_transformer_model.pth')
    
    # Save checkpoint
    if (epoch + 1) % CHECKPOINT_EVERY == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_loss': best_loss,
        }, checkpoint_path)
        print(f"💾 Checkpoint saved at epoch {epoch + 1}")
    
    # Estimate remaining time
    elapsed = time.time() - total_start_time
    eta = (elapsed / (epoch - start_epoch + 1)) * (EPOCHS - epoch - 1) / 60
    
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {avg_loss:.4f} | LR: {scheduler.get_last_lr()[0]:.6f} | Time: {epoch_time:.1f}s | ETA: {eta:.1f}min")

total_time = time.time() - total_start_time
print(f"\n✅ Training complete!")
print(f"📊 Best loss: {best_loss:.4f}")
print(f"⏱️ Total time: {total_time/3600:.2f} hours")

In [12]:
# 📈 Evaluation
model.load_state_dict(torch.load('best_transformer_model.pth'))
model.eval()

y_true, y_pred = [], []
with torch.no_grad():
    for X_batch, lengths_batch, y_batch in test_loader:
        X_batch, lengths_batch = X_batch.to(device), lengths_batch.to(device)
        
        with autocast():
            outputs = model(X_batch, lengths_batch)
        
        preds = torch.sigmoid(outputs).cpu().numpy()
        preds = (preds > 0.5).astype(int)
        y_true.append(y_batch.numpy())
        y_pred.append(preds)

y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

print(f"📊 F1-score (macro): {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"📊 F1-score (micro): {f1_score(y_true, y_pred, average='micro'):.4f}")

SyntaxError: invalid syntax (1764566775.py, line 2)

In [6]:
# 🔮 Prediction Function
def predict(text):
    model.eval()
    tokens = word_tokenize(text, keep_whitespace=False)
    ids = encode_text(tokens, vocab)[:MAX_LEN]
    
    # Pad
    padded_ids = ids + [0] * (MAX_LEN - len(ids))
    
    lengths = torch.tensor([len(ids)], dtype=torch.long).to(device)
    padded = torch.tensor([padded_ids], dtype=torch.long).to(device)
    
    with torch.no_grad():
        with autocast():
            output = model(padded, lengths)
        probs = torch.sigmoid(output).cpu().numpy()[0]
        
        # Multi-label results
        results = [(label_cols[i], float(probs[i])) 
                   for i in range(len(probs)) if probs[i] > 0.5]
        
        if not results:
            best_idx = np.argmax(probs)
            results = [(label_cols[best_idx], float(probs[best_idx]))]
        
        return results

# Test
print(predict("รัฐบาลไทยประกาศนโยบายด้านสิ่งแวดล้อมใหม่"))
print(predict("แรงงานประท้วงเพื่อสิทธิในการทำงาน"))

NameError: name 'model' is not defined

In [ ]:
# 💾 Download model (for Colab)
# from google.colab import files
# files.download('best_transformer_model.pth')